# Konvolučné siete v PyTorchi

Na poslednej prednáške sme si vysvetlili, čo je to konvolúcia a pooling, ako tieto operácie fungujú, a na čo sa používajú v kontexte neurónových sietí. Cieľom tohto cvičenia je natrénovať jednoduché neurónové siete na klasifikáciu obrázkov rukou písaných číslic. Využijeme pritom framework *PyTorch*, ktorý slúži na trénovanie neurónových sietí. Ak ho ešte nemáte nainštalovaný, [urobte tak podľa aktuálneho návodu](https://pytorch.org/get-started/locally/). Následne si stiahnite tento notebook, a pracujte v ňom lokálne, alebo napríklad na [Google Colabe](https://colab.research.google.com).

Ak notebook spúšťate cez Colab, tak pre prácu s grafickou kartou spustite nasledovný kód. Dataset je však jednoduchý, takže ak nemáte nastavenú grafiku na počítači, stále viete sieť natrénovať za niekoľko minút.

In [ ]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

## Načítanie datasetu

Pred tým, než si pripravíme štruktúru neurónovej siete, potrebujeme si načítať trénovacie a testovacie údaje. Na dnešnom cvičení použijeme štandardný [dataset MNIST](https://en.wikipedia.org/wiki/MNIST_database), keďže natrénovanie siete na ňom sa stal akousi novodobou verziou programu *Hello, world!*. Dataset obsahuje rukou písané číslice 0-9 vo forme čiernobielych obrázkov s rozmermi 28×28.

![Ukážka MNIST datasetu](https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png)

Pred samotným načítaním údajov si však najprv naimportujeme potrebné knižnice:

In [ ]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial

Hneď prvým potrebným krokom pri načítavaní obrázkov je potreba ich transformácie na tenzorovú reprezentáciu. V PyTorchi každý obrázok je reprezentovaný ako trojrozmerný tenzor s dimenziami `[channels][height][width]`, kde `channels` je počet farebných kanálov (3 pre RGB obrázky, 1 pre čiernobiele). Tieto transformácie zadefinujeme ako zoznam alebo sériu jednoduchých operácií, ktoré sa vykonajú pri načítaní obrázkov.

In [ ]:
# convert each image to tensor format
transform = transforms.Compose([
    transforms.ToTensor()  # convert to tensor
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

Okrem načítania datasetu si vytvoríme aj dataloadery, ktoré budú zodpovedné za dodávanie použitých údajov v batchoch, teda v dávkach. Okrem veľkosti dávok určíme aj spôsob načítavania, tu napríklad sme nastavili náhodné poradie použitia z dôvodu, že bežné neurónové siete sa natrénujú efektívne v prípade, ak sú trénované na časovo nezávislých dátach.

## Definícia neurónovej siete

Ďalším krokom je definícia modelu neurónovej siete. Na úvod pridáme do našej siete iba jednu konvolučnú vrstvu a padding, aby ste videli základnú štruktúru konvolučného bloku. Pre úplnosť implementácie pridáme aj aktivačnú vrstvu, ako aj `Dropout`, aby sme zabránili preučeniu. Štruktúra siete je nasledovná:

1. Prvá skrytá vrstva (vstupná sa vytvorí automaticky) je konvolučná vrstva `Convolution2D`. Vrstva obsahuje 32 príznakových máp, s kernelmi 5×5 a aktivačnou vrstvou ReLU.
2. Ďalšia vrstva je vrstva max poolingu `MaxPooling2D`. Rozmeri filtra sú 2×2.
3. Ďalej použijeme regularizačnú vrstvu `Dropout`, ktorá náhodne vynuluje 20% neurónov vo vrstve aby sa predišlo pretrénovaniu.
4. Pred tým než sa dostaneme do klasifikačnej časti neurónovej siete, musíme výstup predošlých vrstiev upraviť tak, aby s ním dokázali pracovať plne prepojené vrstvy. Na to slúži operácia *flatten*, teda transformácia tenzorovej reprezentácie na vektor, ktorý môže byť spracovaný tradičnými vrstvami.
5. Pokračujeme vrstvou s 128 neurónmi a ďalšou aktivačnou vrstvou ReLU.
6. Výstupná vrstva je daná počtom tried, teda obsahuje 10 neurónov.

Pre definíciu siete vytvoríme podtriedu `nn.Module`:

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (5, 5), padding=0)
        self.fc1 = nn.Linear(32 * 12**2, 128)
        self.fc2 = nn.Linear(128, 10)
            
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

Pri generácii vektorov pomocou metódy `view` sa ako druhý parameter použije hodnota `-1`, čo slúži na automatický výpočet druhého rozmeru a tak zabezpečí dodržanie veľkosti dávky. Vstup do metódy `forward` teda má rozmery `[batch_size][channels][height][width]`, a dostaneme výsledok s rozmermi `[batch_size][num_classes=10]`.

**Aké sú rozmery vstupov a výstupov jednotlivých vrstiev?**

## Trénovanie siete

Po definícii modelu vieme našu sieť natrénovať. Použijeme na to chybovú funkciu krížovú entropiu (cross-entropy) a ADAM optimalizátor. Trénovať budeme po 5 epochách s veľkosťou batchu 128. Pre jednoduchší zápis trénovania a vyhodnotenia môžeme využiť `torchbearer`.

In [ ]:
# build the model
model = SimpleCNN()

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=5)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

## Rozšírenie siete

Vytvorte rozšírenú štruktúru neurónovej siete s nasledovnou štruktúrou:

1. Konvolučná vrstva s 30 príznakovými mapami s veľkosťou 5×5 a aktiváciou ReLU.
2. Max pooling vrstva s rozmermi 2×2.
3. Konvolučná vrstva s 15 príznakovými mapami s veľkosťou 3×3 a aktiváciou ReLU.
4. Max pooling vrstva s rozmermi 2×2.
5. Dropout vrstva s pravdepodobnosťou 20%.
6. Flatten vrstva.
7. Plne prepojená vrstva s 128 neurónmi a aktiváciou ReLU.
8. Plne prepojená vrstva s 50 neurónmi a aktiváciou ReLU.
9. Výstupná vrstva.

In [ ]:
import torch 
import torch.nn.functional as F
from torch import nn

# Model Definition
class BetterCNN(nn.Module):
    def __init__(self):
        super(BetterCNN, self).__init__()
        # TODO: define layers
    
    def forward(self, x):
        # TODO: define structure
        return

Na trénovanie použite nasledovný kód:

In [ ]:
#reset the data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

# build the model
model = BetterCNN()

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=5)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

## Uloženie siete

Po natrénovaní siete často potrebujete nastavené váhy uložiť a načítať pri ďalšom použití. V PyTorchi sa na ukladávanie modelov používa metóda `torch.save(state, filepath)`, ktorá uloží hodnotu váh do súboru, aby mohla byť načítaná neskôr.

In [ ]:
#save the trained model weights
torch.save(model.state_dict(), "./bettercnn.weights")

Ak pracujete v Colabe, tak si model môžete stiahnuť pomocou metódy:

In [ ]:
from google.colab import files
files.download('bettercnn.weights')

## Používanie siete

Ak máte sieť natrénovanú a uloženú, pravdepodobne ju chcete využívať na predikciu hodnôt počas života modelu, napríklad v rámci aplikácie. V ideálnom prípade by mala byť sieť schopná spracovať vstupy, ktoré sa nenachádzali v pôvodnom datasete, avšak majú rovnaké základné charakteristiky. Na overenie funkčnosti vášho riešenia z predošlého kroku si stiahnite [ukážkové obrázky s číslicami](lab03/imgs.zip).

Následne si načítajte uložené váhy. **Poznámka:** PyTorch defaultne neukladáva štruktúru siete, iba váhy, práve preto ak kód píšete vo viacerých súboroch, triedu definujúcu sieť často potrebujete nakopírovať/naimportovať aj do súboru, v ktorom riešite načítavanie váh.

In [ ]:
import matplotlib.pyplot as plt

# build the model and load state
model = BetterCNN()
model.load_state_dict(torch.load('bettercnn.weights'))

# put model in eval mode
model = model.eval()

Na poslednom riadku predošlého bloku kódu zapneme tzv. *eval* mód, ktorý nastaví sieť na využívanie pre inferenciu. Teda v tomto móde hodnoty váh sa nemenia, a tiež sa deaktivujú niektoré vrstvy, ako napríklad `Dropout`, prípadne `BatchNorm2D`.

Následne si načítame a vizualizujeme ukážkový vstup (môžete si vybrať aj inú ukážku):

In [ ]:
from PIL import Image
import torchvision

transform = torchvision.transforms.ToTensor()
im = transform(Image.open("imgs/1.png"))

plt.imshow(im[0], cmap=plt.get_cmap('gray'))

Teraz už môžeme model využiť na predikciu, avšak sieť očakáva batch na vstupe, kým my dodávame iba jeden obrázok. Práve preto zavoláme metódu `unsqueeze(0)`, ktorá pridá potrebné dimenzie. Výstupom siete je potom 10 hodnôt, ktoré reprezentujú mieru predikcie pre jednotlivé triedy. Finálna predikcia siete je trieda s najvyššou predikovanou hodnotou.

In [ ]:
batch = im.unsqueeze(0)
predictions = model(batch)

print("logits:", predictions.data)

_, predicted_class = predictions.max(1)

print("predicted class:", predicted_class.item())

**Skontrolujte predikciu siete na všetkých ukážkových dátach.**

In [ ]:
transform = torchvision.transforms.ToTensor()
for i in range(10):
    im = transform(Image.open("imgs/{}.png".format(i)))

    plt.imshow(im[0], cmap=plt.get_cmap('gray'))
    batch = im.unsqueeze(0)
    predictions = model(batch)

    print("logits:", predictions.data)

    _, predicted_class = predictions.max(1)

    print("predicted class:", predicted_class.item())

## Vizualizácia filtrov

Na to, aby sme získali intuitívny pohľad do fungovania neurónovej siete, môžeme vizualizovať rôzne charakteristiky jednotlivých vrstiev a filtrov. Filtre vieme vizualizovať vďaka tomu, že ich môžeme považovať za malé obrázky, ktoré popisujú základné črty vstupov. Váhy filtrov vieme načítať priamo z natrénovanej siete a vizualizovať ich pomocou `matplotlib`:

In [ ]:
weights = model.conv1.weight.data.cpu()

# plot the first layer features
for i in range(0,30):
    plt.subplot(5,6,i+1)
    plt.imshow(weights[i, 0, :, :], cmap=plt.get_cmap('gray'))
plt.show()

Hodnota `model.conv1.data` je tenzor s hodnotami váh, metóda `cpu()` načíta dáta z GPU do procesora pre lepšiu vizualizáciu. Vzhľadom na to, že vplyv jednotlivých váh sa agreguje ako postupujeme v sieti, veľmi často dáva zmysel vizualizovať iba filtre prvej vrstvy.

## Vizualizácia príznakových máp

Podobným spôsobom ako samotné filtre dokážeme vykresliť aj ich efekt na vybraný vstup tak, že vstup preženieme sieťou a získame výstupy na ľubovoľnej vrstve. V PyTorchi to vieme urobiť použitím tzv. `hook` objektu, ktorý slúži na pozastavenie vykonávania dopredného chodu na istej vrstve. Napríklad pre druhú konvolučnú vrstvu môžeme použiť:

In [ ]:
transform = torchvision.transforms.ToTensor()
im = transform(Image.open("imgs/1.PNG")).unsqueeze(0)

def hook_function(module, grad_in, grad_out):
    for i in range(grad_out.shape[1]):
        conv_output = grad_out.data[0, i]
        plt.subplot(5, int(1+grad_out.shape[1]/5), i+1)
        plt.imshow(conv_output, cmap=plt.get_cmap('gray'))
        
hook = model.conv2.register_forward_hook(hook_function) # register the hook
model(im) # forward pass
hook.remove() #Tidy up

**Vyskúšajte vizualizáciu pre výstupy prvej konvolučnej vrstve.**

## Vizualizácia maximálnej aktivácie

Posledným užitočným spôsobom vizualizácie toho, čo sa filtre naučia, je nájsť vstupný obrázok, ktorý by spôsobil maximálnu aktiváciu filtra. Takýto obrázok vieme získať tak, že vygenerujeme náhodný šum, ktorý gradientovým vzostupom optimalizujeme dovtedy, kým nenájdeme maximalizáciu pre daný filter. Nasledovný kód získa presne takýto obrázok:

In [ ]:
def visualise_maximum_activation(model, target, num=10, alpha = 1.0):
    for selected in range(num):
        input_img = torch.randn(1, 1, 28, 28, requires_grad=True)

        # we're interested in maximising outputs of the 3rd layer:
        conv_output = None

        def hook_function(module, grad_in, grad_out):
            nonlocal conv_output
            # Gets the conv output of the selected filter/feature (from selected layer)
            conv_output = grad_out[0, selected]

        hook = target.register_forward_hook(hook_function)

        for i in range(30):
            model(input_img)
            loss = torch.mean(conv_output)
            loss.backward()

            norm = input_img.grad.std() + 1e-5
            input_img.grad /= norm
            input_img.data = input_img + alpha * input_img.grad

        hook.remove()

        input_img = input_img.detach()

        plt.subplot(2,num/2,selected+1)
        plt.imshow(input_img[0,0], cmap=plt.get_cmap('gray'))

    plt.show()
    
visualise_maximum_activation(model, model.fc3)